 <h1>add required imports and constants</h1>

In [9]:
np.__version__

'1.15.0'

In [1]:
%load_ext line_profiler

import json
import os
import os.path as path
import random
import six



from annoy import AnnoyIndex
import numpy as np
from torchvision import transforms as torch_transform
import torch
from torch.autograd import Variable
import torch.nn.functional as F
from PIL import Image
from torch.utils.data import Dataset
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision.models as models
from PIL import ImageFile




PHOTO_FOLDER = "../data/images"
META_FOLDER = "../data/"
DRESS_FOLDER = "../data/json/"


ID_NAME = "retrieval_dresses.json"
TEST_NAME = "test_pairs_dresses.json"
TRAIN_NAME = "train_pairs_dresses.json"
PHOTO_FILE = "../data/photos/photos.txt"
PHOTO_DRESS_FILE = "../data/photos/photos_dress.txt"
SEED = 17
EPOCH = 100
MARGIN = 0.3
CHECKPOINT_FILE = "checkpoint.pth.tar"
IMG_SIZE =224
BATCH_SIZE = 10
HAS_CUDA = torch.cuda.is_available()
LR = 0.0001



torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
ImageFile.LOAD_TRUNCATED_IMAGES = True
FINAL_EMBEDDING_SIZE = 128

def read_json(file_name):
    with open(file_name) as f:
        obj = json.loads(f.readline())
    return obj

<h1>download dataset metadata and photos info(optional)</h1>

In [59]:
!git clone https://github.com/pumpikano/street2shop

Cloning into 'street2shop'...
remote: Counting objects: 9, done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 9 (delta 1), reused 9 (delta 1), pack-reused 0
Unpacking objects: 100% (9/9), done.


In [4]:
!mkdir street2shop/images

In [10]:
!curl -O http://www.tamaraberg.com/street2shop/wheretobuyit/meta.zip
!curl http://www.tamaraberg.com/street2shop/wheretobuyit/photos.tar | tar x

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2777k  100 2777k    0     0   297k      0  0:00:09  0:00:09 --:--:--  346k


In [13]:
!unzip meta.zip
!rm meta.zip

Archive:  meta.zip
   creating: meta/
  inflating: meta/.DS_Store          
   creating: __MACOSX/
   creating: __MACOSX/meta/
  inflating: __MACOSX/meta/._.DS_Store  
   creating: meta/json/
  inflating: meta/json/.DS_Store     
   creating: __MACOSX/meta/json/
  inflating: __MACOSX/meta/json/._.DS_Store  
  inflating: meta/json/retrieval_bags.json  
  inflating: meta/json/retrieval_belts.json  
  inflating: meta/json/retrieval_dresses.json  
  inflating: meta/json/retrieval_eyewear.json  
  inflating: meta/json/retrieval_footwear.json  
  inflating: meta/json/retrieval_hats.json  
  inflating: meta/json/retrieval_leggings.json  
  inflating: meta/json/retrieval_outerwear.json  
  inflating: meta/json/retrieval_pants.json  
  inflating: meta/json/retrieval_skirts.json  
  inflating: meta/json/retrieval_tops.json  
  inflating: meta/json/test_pairs_bags.json  
  inflating: meta/json/test_pairs_belts.json  
  inflating: meta/json/test_pairs_dresses.json  
  inflating: meta/json/test_pai

In [18]:
!mkdir ../data
!mkdir ../data/images

mkdir: cannot create directory ‘../data/images’: File exists


In [1]:
def get_photo_ids(obj, retr_dict):
    photos = set()
    for item in obj:
        photos.add(item["photo"])
        product_photos = retr_dict[item["product"]]
        photos.update(product_photos)
    return photos

In [30]:
if __name__ == "__main__":
    id_to_photo_dirty = read_json(DRESS_FOLDER+ID_NAME)
    id_to_photo_clean = {}
    for item in id_to_photo_dirty:
        if item["product"] in id_to_photo_clean:
            id_to_photo_clean[item["product"]].append(item["photo"])
        else:
            id_to_photo_clean[item["product"]] = [item["photo"]]
    test_dress = read_json(DRESS_FOLDER + TEST_NAME)
    train_dress = read_json(DRESS_FOLDER + TRAIN_NAME)
    test_set = get_photo_ids(test_dress, id_to_photo_clean)
    train_set = get_photo_ids(train_dress, id_to_photo_clean)
    result_set = test_set|train_set

    with open(PHOTO_DRESS_FILE, "w") as new_ph_file:
        with open(PHOTO_FILE) as ph_file:
            for line in ph_file:
                id = int(line.split(",")[0])
                if id in result_set:
                    new_ph_file.write(line)

In [40]:
!python street2shop/download.py --urls photos/photos_dress.txt --image_dir ../../data/images

^C
Error in atexit._run_exitfuncs:
Traceback (most recent call last):
  File "/home/snake/anaconda3/envs/myenv/lib/python3.6/concurrent/futures/thread.py", line 40, in _python_exit
    t.join()
  File "/home/snake/anaconda3/envs/myenv/lib/python3.6/threading.py", line 1056, in join
    self._wait_for_tstate_lock()
  File "/home/snake/anaconda3/envs/myenv/lib/python3.6/threading.py", line 1072, in _wait_for_tstate_lock
    elif lock.acquire(block, timeout):
KeyboardInterrupt


<h1> prepare datasets </h1>

In [5]:
ALL_IMAGES  = [int(name.split(".")[0]) for name in os.listdir(PHOTO_FOLDER) if path.isfile(PHOTO_FOLDER+"/"+name)]

In [15]:
def default_image_loader(path, crop=None,size=None):
    if size is None:
        size = (IMG_SIZE, IMG_SIZE)
    image=Image.open(path).convert('RGB')
    if crop is not None:
        image = image.crop(crop)
    image=image.resize(size)
    return image

def get_negative(positive, items):
    while True:
        rand_item = random.choice(items)
        if positive != rand_item:
            return rand_item
    

def get_image(photo_id, folder=PHOTO_FOLDER, crop=None,transform=None):
    photo_name = str(photo_id)
    zeros = "0"*(9- len(photo_name))
    extension = ".jpeg", ".png", ".gif"
    full_photo_name = None
    for ext in extension:
        full_photo_name = folder+"/"+zeros+photo_name+ext
        if path.isfile(full_photo_name):
            break
    else:
        raise Exception("No Image found")
    image = default_image_loader(full_photo_name, crop=crop)
    if transform is None:
        return image
    else:
        return transform(image)

class DressDataset(Dataset):
    """street2shop dress dataset."""
    
    cach = {}
    
    def __init__(self, photos_folder_name=PHOTO_FOLDER, transform = None):
        self.triplets = None
        self.retrieval = None
        self.photos_folder_name = photos_folder_name
        self.transform = transform
    
    def parse_triplets(self, triplets_file_name, retrieval_file_name=DRESS_FOLDER+ID_NAME):
        self.triplets = read_json(triplets_file_name)
        self.retrieval = read_json(retrieval_file_name)
        self.retrieval ={it["product"]:it["photo"] for it in self.retrieval}
        all_photos = [item["photo"] for item in self.triplets]
        for item in self.triplets:
            positive_photo = self.retrieval[item["product"]]
            anchor_photo = item["photo"]
            if positive_photo not in ALL_IMAGES or anchor_photo not in ALL_IMAGES:
                continue
            item["positive"] = positive_photo
            item["anchor"] = anchor_photo
            del item["product"]
            del item["photo"]
            negative = get_negative(item["positive"], ALL_IMAGES)
            item["negative"] = negative
        
        for item in self.triplets[:]:
            if "positive" not in item:
                self.triplets.remove(item)
    
    def __len__(self):
        return len(self.triplets)

    def __getitem__(self, index):
        sample = {}
        triplet = self.triplets[index]
        sample["positive"] = get_image(triplet["positive"], transform = self.transform)
        sample["negative"] = get_image(triplet["negative"], transform = self.transform)
        bbox = triplet["bbox"]
        anchor = get_image(triplet["anchor"], crop=(bbox["left"],bbox["top"],bbox["left"]+bbox["width"],bbox["top"]+bbox["height"]),
                          transform=self.transform)
        sample["anchor"] = anchor
        return ((sample["positive"],triplet["positive"]), (sample["anchor"],triplet["anchor"]),
                (sample["negative"], triplet["negative"]))
    
    def get_sliced_copy(self, start, end):
        copied = DressDataset(photos_folder_name=self.photos_folder_name, transform=self.transform)
        copied.triplets = self.triplets[start:end]
        return copied
    
    def resample(self, embeddings, mapping, margin):
        new_triplets = []
        reverse_mapping = {v:k for k,v in six.iteritems(mapping)}
        for triplet in self.triplets:
            pos, anc, neg = mapping[triplet["positive"]], mapping[triplet["anchor"]], mapping[triplet["negative"]]
            dists = np.linalg.norm(embeddings - embeddings[anc], axis = 1)
            delta = dists[pos] - dists
            semihard_cands = np.squeeze(np.where(np.logical_and(delta>0, delta<margin)))
            if semihard_cands.size:
                while True:
                    neg_cand = np.random.choice(semihard_cands)
                    if neg_cand not in (pos, anc):
                        break
            else:
                neg_cand = neg
            triplet["negative"] = reverse_mapping[neg_cand]
            new_triplets.append(triplet)
        
        self.triplets = new_triplets
    
    def state_dict(self):
        state = {}
        keys = ["photos_folder_name", "triplets","retrieval"]
        for k in keys:
            state[k] = getattr(self,k)
        return state
    
    @classmethod
    def load(cls, state_dict, transform):
        instance = cls(photos_folder_name=state_dict["photos_folder_name"], transform=transform)
        instance.retrieval = state_dict["retrieval"]
        instance.triplets = state_dict["triplets"]
        return instance
                                                     
    @classmethod
    def get_parsed_dataset(cls, triplets_file_name, photos_folder_name=PHOTO_FOLDER, 
                 retrieval_file_name=DRESS_FOLDER+ID_NAME, transform = None):
        ds = DressDataset(photos_folder_name=photos_folder_name, transform=transform)
        ds.parse_triplets(triplets_file_name=triplets_file_name, retrieval_file_name=retrieval_file_name)
        return ds  
        
        

<h1> dataset testing</h1>

In [16]:
fake_train = DressDataset.get_parsed_dataset(triplets_file_name=DRESS_FOLDER+TRAIN_NAME,
                     transform=torch_transform.Compose([
                                               torch_transform.ToTensor()]))

In [18]:
%%time
fake_mappings = {}
counter=0
for triplet in fake_train.triplets:
            pos, anc, neg = triplet["positive"], triplet["anchor"], triplet["negative"]
            if pos not in fake_mappings:
                fake_mappings[pos] = counter
                counter = counter+1
            if anc not in fake_mappings:
                fake_mappings[anc] = counter
                counter = counter+1
            if neg not in fake_mappings:
                fake_mappings[neg] = counter
                counter = counter+1
if HAS_CUDA:
    fake_embeddings = torch.randn(len(fake_mappings),FINAL_EMBEDDING_SIZE).cpu().data.numpy()
else:
    fake_embeddings = torch.randn(len(fake_mappings),FINAL_EMBEDDING_SIZE).numpy()
%lprun -f fake_train.resample fake_train.resample(fake_embeddings, fake_mappings, margin=0.3)

/home/snake/anaconda3/envs/myenv/lib/python3.6/site-packages/numpy/linalg/linalg.py:2378: RuntimeWarning: invalid value encountered in sqrt
  return sqrt(add.reduce(s, axis=axis, keepdims=keepdims))


CPU times: user 1min 55s, sys: 968 ms, total: 1min 56s
Wall time: 30.5 s


In [21]:
train = DressDataset.get_parsed_dataset(triplets_file_name=DRESS_FOLDER+TRAIN_NAME,
                     transform=torch_transform.Compose([
                                               torch_transform.ToTensor()
                                           ]))
test = DressDataset.get_parsed_dataset(triplets_file_name=DRESS_FOLDER+TRAIN_NAME,
                   transform=torch_transform.Compose([
                                               torch_transform.ToTensor()
                                           ]))

In [22]:
micro_train  = train.get_sliced_copy(0,500)

In [23]:
VALIDATION_PROPORTION = 0.5
val_prop = int(len(test)*VALIDATION_PROPORTION)
validation = test.get_sliced_copy(0, val_prop)
final_test = test.get_sliced_copy(val_prop,len(test))

<h1>define the model</h1>

In [24]:
class BasisNet(nn.Module):
    def __init__(self, embedding_size=FINAL_EMBEDDING_SIZE):
        super(BasisNet, self).__init__()
        self.inception = models.resnet18(pretrained=True)
        for param in self.inception.parameters():
            param.requires_grad=False
        num_ftrs = self.inception.fc.in_features
        self.inception.fc = nn.Linear(num_ftrs, 512)
        self.elu = nn.ELU()
        self.final = nn.Linear(512, embedding_size)
    
    def forward(self,x):
        x = self.inception(x)
        x = self.elu(x)
        x = self.final(x)
        return self.l2_norm(x)
    
    def l2_norm(self,inp):
        input_size = inp.size()
        buffer = torch.pow(inp, 2)

        normp = torch.sum(buffer, 1).add_(1e-10)
        norm = torch.sqrt(normp)

        _output = torch.div(inp, norm.view(-1, 1).expand_as(inp))

        output = _output.view(input_size)

        return output
        
class TripletNet(nn.Module):
    def __init__(self, embedding_net):
        super(TripletNet, self).__init__()
        self.embedding_net = embedding_net

    def forward(self, positive, anchor, negative):
        output1 = self.embedding_net(positive)
        output2 = self.embedding_net(anchor)
        output3 = self.embedding_net(negative)
        return output1, output2, output3

    def get_embedding(self, x):
        return self.embedding_net(x)

In [25]:
if HAS_CUDA:
    cuda = torch.device('cuda') 

In [26]:
def loss_func(anchor, positive, negative, size_average=True, margin = 0.3):
    distance_positive = (anchor - positive).pow(2).sum(1)  # .pow(.5)
    distance_negative = (anchor - negative).pow(2).sum(1)  # .pow(.5)
    losses = F.relu(distance_positive - distance_negative + margin)
    return losses.mean() if size_average else losses.sum()

class TripletLoss(nn.Module):
    """
    Triplet loss
    Takes embeddings of an anchor sample, a positive sample and a negative sample
    """

    def __init__(self, margin):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative, size_average=True):
        return loss_func(anchor, positive, negative, size_average, self.margin)
    
dress_net = TripletNet(BasisNet())
if HAS_CUDA:
    dress_net.to(cuda)

In [27]:
def save_embedding(mapping, embeddings, emb_to_save, index, annoy):
    if index not in mapping:
        embeddings.append(emb_to_save)
        new_index = len(embeddings) - 1
        mapping[index] = new_index
        if HAS_CUDA:
            annoy.add_item(new_index, emb_to_save.cpu().data.numpy())
        else:
            annoy.add_item(new_index, emb_to_save.numpy())
        return new_index
    else:
        return mapping[index]

In [28]:
def save_checkpoint(state, filename=CHECKPOINT_FILE):
    torch.save(state, filename)

In [37]:
def fit(model, train_dataset, batch_size=BATCH_SIZE, lr=LR, margin=MARGIN, n_epoch=EPOCH, save=False, validation_dataset = None):
    optimizer = optim.Adam(filter(lambda p: p.requires_grad,model.parameters()), lr=lr)
    kwargs = {'num_workers': 10, 'pin_memory': True} if HAS_CUDA else {}
    loader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size, shuffle=False,**kwargs)
    all_losses=[]
    criterion = TripletLoss(margin)
    for epoch in range(n_epoch):
        losses = []
        train_embeddings=[]
        train_embedding_map = {}
        torch.set_grad_enabled(False)
        annoy_index = AnnoyIndex(FINAL_EMBEDDING_SIZE, metric="euclidean")
        for batch_i, (positive, anchor, negative) in enumerate(loader):
            if HAS_CUDA:
                vpositive, vanchor, vnegative = Variable(positive[0].pin_memory()).cuda(async=True), Variable(anchor[0].pin_memory()).cuda(async=True), Variable(negative[0].pin_memory()).cuda(async=True)
            else:
                vpositive, vanchor, vnegative = Variable(positive[0]), Variable(anchor[0]), Variable(negative[0])
            p_emb,a_emb,n_emb = dress_net(vanchor, vpositive, vnegative)
            for i in range(len(p_emb)):
                save_embedding(train_embedding_map, train_embeddings, p_emb[i],positive[1][i].item(), annoy_index)
                save_embedding(train_embedding_map, train_embeddings, a_emb[i],anchor[1][i].item(), annoy_index)
                save_embedding(train_embedding_map, train_embeddings, n_emb[i],negative[1][i].item(), annoy_index)
        if HAS_CUDA:
            train_embeddings_ct = torch.stack(train_embeddings).cpu().data.numpy()
        else:
            train_embeddings_ct = torch.stack(train_embeddings).numpy()
        train_dataset.resample(train_embeddings_ct, train_embedding_map,
                       margin = margin)
        torch.set_grad_enabled(True)
        for batch_i, (positive, anchor, negative) in enumerate(loader):
            if HAS_CUDA:
                vpositive, vanchor, vnegative = Variable(positive[0].cuda(async=True)), Variable(anchor[0].cuda(async=True)), Variable(negative[0].cuda(async=True))
            else:
                vpositive, vanchor, vnegative = Variable(positive[0]), Variable(anchor[0]), Variable(negative[0])
            p_emb,a_emb,n_emb = dress_net(vanchor, vpositive, vnegative)
            loss = criterion(p_emb,a_emb,n_emb)
            losses.append(loss.item())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        score = np.mean(losses)
        all_losses.append(score)
        print("loss %s after epoch %s "%(score, epoch))
        if save:
            save_checkpoint({
                'epoch': epoch + 1,
                'model': dress_net.state_dict(),
                'dataset': micro_train.state_dict(),
                'optimizer' : optimizer.state_dict(),
            })
        
        if validation_dataset is not None:
            torch.set_grad_enabled(False)
            pairs = []
           
            val_loader = torch.utils.data.DataLoader(validation_dataset,batch_size=batch_size, shuffle=True,**kwargs)
            for batch_i, (positive, anchor, negative) in enumerate(val_loader):
                if HAS_CUDA:
                    vpositive, vanchor, vnegative = Variable(positive[0].cuda(async=True)), Variable(anchor[0].cuda(async=True)), Variable(negative[0].cuda(async=True))
                else:
                    vpositive, vanchor, vnegative = Variable(positive[0]), Variable(anchor[0]), Variable(negative[0])
                p_emb,a_emb,n_emb = dress_net(vanchor, vpositive, vnegative)
                for i in range(len(p_emb)):
                    pair1 = save_embedding(train_embedding_map, train_embeddings, p_emb[i],positive[1][i].item(), annoy_index)
                    pair2 = save_embedding(train_embedding_map, train_embeddings, a_emb[i],anchor[1][i].item(), annoy_index)
                    pairs.append((pair1, pair2))
            annoy_index.build(50)
            accuracy = 0.0
            for pair in pairs:
                neighbors = annoy_index.get_nns_by_item(pair[0], 10)
                if pair[1] in neighbors:
                    accuracy= accuracy+1
            total = len(validation_dataset)
            print(" top 5 accuracy %s"% str(accuracy/total))

In [38]:
%lprun -f fit fit(dress_net,micro_train, n_epoch=1, validation_dataset=test.get_sliced_copy(0,100), save =True)

loss 0.053696143282577394 after epoch 0 
 top 5 accuracy 0.6


In [39]:
%%time
fit(dress_net,train, n_epoch=5, validation_dataset=test.get_sliced_copy(0,1000))

/home/snake/anaconda3/envs/myenv/lib/python3.6/site-packages/PIL/Image.py:2546: DecompressionBombWarning: Image size (123871510 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/home/snake/anaconda3/envs/myenv/lib/python3.6/site-packages/PIL/Image.py:2546: DecompressionBombWarning: Image size (123871510 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/home/snake/anaconda3/envs/myenv/lib/python3.6/site-packages/numpy/linalg/linalg.py:2378: RuntimeWarning: invalid value encountered in sqrt
  return sqrt(add.reduce(s, axis=axis, keepdims=keepdims))
/home/snake/anaconda3/envs/myenv/lib/python3.6/site-packages/PIL/Image.py:2546: DecompressionBombWarning: Image size (123871510 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/home/snake/anaconda3/envs/myenv/lib/python3.6/site-packages/PIL/Image.py:2546: Decompr

loss 0.09411124714362466 after epoch 0 
 top 5 accuracy 0.164


/home/snake/anaconda3/envs/myenv/lib/python3.6/site-packages/PIL/Image.py:2546: DecompressionBombWarning: Image size (123871510 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/home/snake/anaconda3/envs/myenv/lib/python3.6/site-packages/PIL/Image.py:2546: DecompressionBombWarning: Image size (123871510 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/home/snake/anaconda3/envs/myenv/lib/python3.6/site-packages/PIL/Image.py:2546: DecompressionBombWarning: Image size (123871510 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/home/snake/anaconda3/envs/myenv/lib/python3.6/site-packages/PIL/Image.py:2546: DecompressionBombWarning: Image size (123871510 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/home/snake/anaconda3/envs/myenv/lib/python3.6/s

loss 0.08068024022037779 after epoch 1 
 top 5 accuracy 0.112


/home/snake/anaconda3/envs/myenv/lib/python3.6/site-packages/PIL/Image.py:2546: DecompressionBombWarning: Image size (123871510 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/home/snake/anaconda3/envs/myenv/lib/python3.6/site-packages/PIL/Image.py:2546: DecompressionBombWarning: Image size (123871510 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/home/snake/anaconda3/envs/myenv/lib/python3.6/site-packages/PIL/Image.py:2546: DecompressionBombWarning: Image size (123871510 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/home/snake/anaconda3/envs/myenv/lib/python3.6/site-packages/PIL/Image.py:2546: DecompressionBombWarning: Image size (123871510 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


loss 0.0697899733054523 after epoch 2 
 top 5 accuracy 0.118


/home/snake/anaconda3/envs/myenv/lib/python3.6/site-packages/PIL/Image.py:2546: DecompressionBombWarning: Image size (123871510 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/home/snake/anaconda3/envs/myenv/lib/python3.6/site-packages/PIL/Image.py:2546: DecompressionBombWarning: Image size (123871510 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


loss 0.0617109383507752 after epoch 3 
 top 5 accuracy 0.115


/home/snake/anaconda3/envs/myenv/lib/python3.6/site-packages/PIL/Image.py:2546: DecompressionBombWarning: Image size (123871510 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/home/snake/anaconda3/envs/myenv/lib/python3.6/site-packages/PIL/Image.py:2546: DecompressionBombWarning: Image size (123871510 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/home/snake/anaconda3/envs/myenv/lib/python3.6/site-packages/PIL/Image.py:2546: DecompressionBombWarning: Image size (123871510 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/home/snake/anaconda3/envs/myenv/lib/python3.6/site-packages/PIL/Image.py:2546: DecompressionBombWarning: Image size (123871510 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


loss 0.056248353809449814 after epoch 4 
 top 5 accuracy 0.139
CPU times: user 12min 56s, sys: 4min 38s, total: 17min 35s
Wall time: 1h 6min 43s


In [67]:
saved_net = TripletNet(BasisNet())
saved_net.to(cuda)
saved_optimizer = optim.Adam(filter(lambda p: p.requires_grad,saved_net.parameters()), lr=0.0001)
checkpoint = torch.load('/tmp/checkpoint.pth.tar')
saved_model = saved_net.load_state_dict(checkpoint["model"])
saved_optimizer = saved_optimizer.load_state_dict(checkpoint["optimizer"])
saved_ds = DressDataset.load(checkpoint["dataset"], transform = train.transform)